# SQL football data exploration project

Football has come a long way since its inception, and modern-day football wouldn't exist without big data analysis. With that thought in mind, I decided to explore a dataset from Kaggle regarding European football. It consists of 11 different European leagues, with the top five leagues included, spanning a time-period from the 2008/2009 season to the 2015/2016 season. To complete this project I used DB Browser for SQLite as well as Jupyter Notebook and some Python/Pandas coding. This portfolio project has enhanced my data analysis skills and I hope that by exploring this dataset it will give me insights into big data in the football world.

Firstly, I downloaded the European Soccer Database from Kaggle. There were nine tables, including leagues, matches, and teams etc. Some of them had 25,000+ rows, but some of them contained basic data, like in a "country" table. Obviously, data (although this was a great and very well organized database) needed some cleaning. What is more, for the purpose of exploring the data, I needed to manipulate the data. This is why, for example, I changed the date format in the Match table. Originally the date format was written with the hour, but all of the games started at midnight (00:00:00), which is obviously not true. Therefore, I used the UPDATE and REPLACE statements in order to eliminate hour from the date format. I also altered tables, added columns called "Score" and "Result" and added data, mainly based on columns named "home_team_goal" and "away_team_goal". This format was not bad, but without Result or Score it would have been difficult to analyse football data. I also used the CONCATENATE statement; however, in SQLite it is used differently than with other SQL tools. To summarize, I put the query that I used below. For the purpose of this project, I put it as text; I will only show the output of the SELECT statement later.

Update Match 
Set date = Replace(date, '00:00:00', '');

Alter table Match
Add column result text;

Update Match
Set Result = '1'
Where home_team_goal > away_team_goal;

Update Match
Set Result = '0'
Where home_team_goal = away_team_goal;

Update Match
Set Result = '2'
Where home_team_goal < away_team_goal;

Alter table Match
Add column Score text;

Update Match
Set Score = home_team_goal || ' - ' || away_team_goal


After these operations, we are ready to analyze data. I mainly used the "Match" and "Team" tables. First, we need to import two libraries in order to be able to analyze data using Python: sqlite3 and pandas.

In [2]:
import sqlite3
import pandas as pd

Using only the SQLite3 library is also possible, but the output looks better with the Pandas library, so I used it. I wrote the SQLite3.connect() function to interact with the SQLite database. Next, I used the Pandas library to read the SQL query. Finally, we were ready to write what we needed. The main problem with the "Match" table was that there weren't any team names; I had to join the "Team" table, but this wasn't enough. These tables had to be joined twice, once for the home team data and once for the away team data, in order to know both participants of each match. 

In [3]:
con = sqlite3.connect('football_data.sqlite')
df=pd.read_sql("""SELECT m.date, t.team_long_name AS Home_Team_Name, a.team_long_name AS Away_Team_Name, m.home_team_goal, m.away_team_goal, m.result, m.Score
FROM Match m
JOIN Team t
ON m.home_team_api_id = t.team_api_id
Join Team a
ON m.away_team_api_id = a.team_api_id;""", con=con)
df

,date,Home_Team_Name,Away_Team_Name,home_team_goal,away_team_goal,result,Score
0,2008-08-17,KRC Genk,Beerschot AC,1,1,0,1 - 1
1,2008-08-16,SV Zulte-Waregem,Sporting Lokeren,0,0,0,0 - 0
2,2008-08-16,KSV Cercle Brugge,RSC Anderlecht,0,3,2,0 - 3
3,2008-08-17,KAA Gent,RAEC Mons,5,0,1,5 - 0
4,2008-08-16,FCV Dender EH,Standard de Liège,1,3,2,1 - 3
...,...,...,...,...,...,...,...
25974,2015-09-22,FC St. Gallen,FC Thun,1,0,1,1 - 0
25975,2015-09-23,FC Vaduz,FC Luzern,1,2,2,1 - 2
25976,2015-09-23,Grasshopper Club Zürich,FC Sion,2,0,1,2 - 0
25977,2015-09-22,Lugano,FC Zürich,0,0,0,0 - 0


Now we can have fun with aggregate functions. In football, as in many sports, playing at home is considered a big advantage. Having more than 25,000 matches in this database is a great opportunity to check it, given that the data includes league games, meaning each team played with the same opponent at home and away in one season. An even number of games won at home and away would mean there is no advantage. So, how does it really look? For this purpose, I used the count function and my created column called "Result". Draws are marked as 0, home wins are marked as 1, and away wins are marked as 2.

In [4]:
Results =pd.read_sql("""Select Result, Count(Result)
From Match
Group By Result;""", con=con)
Results

,result,Count(Result)
0,0,6596
1,1,11917
2,2,7466


SQLite is not able to perform math, so changing this output to percentage values was not possible, but I can do this here. So, teams playing at home won 45.9% of games, away - 28.7%, and draws occurred in 25.4% of them. We can consider this as a significant advantage.

Next query is about another created column, which is "Score". First, we're gonna look at the popularity of distinct scores that occured in games:

In [6]:
Dist_scores = pd.read_sql("""Select Distinct Score, Count(Score)
From Match
Group by Score
Order by Count(Score) DESC
LIMIT 15;""", con=con)
Dist_scores

,Score,Count(Score)
0,1 - 1,3014
1,1 - 0,2643
2,2 - 1,2180
3,2 - 0,2132
4,0 - 0,1978
5,0 - 1,1863
6,1 - 2,1659
7,2 - 2,1310
8,3 - 1,1206
9,0 - 2,1205


We see that the most common result is 1-1, and it occurred in more than 3,000 games in this database. Additionally, the most common draws are 0-0 and 2-2. Out of 6,596 draws in this database, 0-0, 1-1, or 2-2 draws happened in 95.6% of all draws. The rest of draws, like 3-3 or 4-4, are much less common.

We can also look for the least common scores. The public loves games where there are a lot of goals, but these games are normally less common. Using this query, I list the games with scores that occur only once in this database.

In [10]:
Crazy_scores = pd.read_sql("""SELECT m.date, t.team_long_name AS Home_Team_Name, a.team_long_name AS Away_Team_Name, m.result, m.Score
FROM Match m
JOIN Team t
ON m.home_team_api_id = t.team_api_id
Join Team a
ON m.away_team_api_id = a.team_api_id
Group by m.Score
Having Count(Score) = 1;""", con=con)
Crazy_scores

,date,Home_Team_Name,Away_Team_Name,result,Score
0,2016-03-13,ES Troyes AC,Paris Saint-Germain,2,0 - 9
1,2011-05-07,FC St. Pauli,FC Bayern Munich,2,1 - 8
2,2010-10-24,PSV,Feyenoord,1,10 - 0
3,2015-12-20,Real Madrid CF,Rayo Vallecano,1,10 - 2
4,2014-10-26,En Avant de Guingamp,OGC Nice,2,2 - 7
5,2014-09-20,RC Deportivo de La Coruña,Real Madrid CF,2,2 - 8
6,2011-11-06,FC Utrecht,Ajax,1,6 - 4
7,2010-05-05,Motherwell,Hibernian,0,6 - 6
8,2010-01-16,Chelsea,Sunderland,1,7 - 2
9,2011-08-28,Manchester United,Arsenal,1,8 - 2


In this database, there is also betting data. This is a good opportunity to look for big upsets in this database. Again, because there are normally not a lot of results like this, I can list the participants of these games, so I joined tables "Match" and "Team". Creating a procedure would be a good way to handle this without writing the same query again, but in SQLite this is no longer possible. 

In [11]:
Upsets = pd.read_sql("""SELECT m.date, t.team_long_name AS Home_Team_Name, a.team_long_name AS Away_Team_Name, m.result, m.Score, m.B365H, m.B365D, m.B365A
FROM Match m
JOIN Team t
ON m.home_team_api_id = t.team_api_id
Join Team a
ON m.away_team_api_id = a.team_api_id
Where m.B365H >= 15 AND m.result = 1 OR m.B365A >=15 AND m.result = 2;""", con=con)
Upsets

,date,Home_Team_Name,Away_Team_Name,result,Score,B365H,B365D,B365A
0,2008-09-27,Arsenal,Hull City,2,1 - 2,1.20,6.00,17.00
1,2010-11-14,Chelsea,Sunderland,2,0 - 3,1.22,5.75,15.00
2,2010-09-25,Arsenal,West Bromwich Albion,2,2 - 3,1.22,6.00,15.00
3,2011-12-31,Manchester United,Blackburn Rovers,2,2 - 3,1.13,8.00,26.00
4,2013-01-02,Chelsea,Queens Park Rangers,2,0 - 1,1.20,6.50,15.00
5,2014-08-30,Manchester City,Stoke City,2,0 - 1,1.18,8.00,18.00
6,2015-03-22,FC Bayern Munich,Borussia Mönchengladbach,2,0 - 2,1.17,7.50,17.00
7,2016-03-02,FC Bayern Munich,1. FSV Mainz 05,2,1 - 2,1.13,9.00,21.00
8,2013-01-06,Juventus,Sampdoria,2,1 - 2,1.18,6.50,16.00
9,2012-11-04,AZ,VVV-Venlo,2,1 - 2,1.20,6.00,15.00


Huge fans of the largest squads may recall several of those outcomes. Barcelona endured a number of significant defeats as a strong favorite, counting the clash against Hercules Club de Futbol. That match stands as the ultimate surprise as Hercules emerged victorious by 2 goals and, for every euro staked, Bet365 paid out 34 euros.

I had the chance to search for goals scored in all fixtures during a single season of certain leagues. I decided to exclude the Belgian Jupiler League since there was missing info about certain contests. It is also important to bear in mind that various leagues have gameweeks that last either 38 or 30 weeks like the Polish Ekstraklasa.

In [16]:
Goals_overall = pd.read_sql("""Select m.season, Sum(m.home_team_goal + m.away_team_goal) AS Goals_Total, l.name
FROM Match m
Join League l
ON m.country_id = l.country_id
Where l.name NOT LIKE 'Belgium Jupiler League' AND season = '2008/2009'
Group by m.season, l.name
Order by Goals_Total DESC;""", con=con)
Goals_overall

,season,Goals_Total,name
0,2008/2009,1101,Spain LIGA BBVA
1,2008/2009,988,Italy Serie A
2,2008/2009,942,England Premier League
3,2008/2009,894,Germany 1. Bundesliga
4,2008/2009,870,Netherlands Eredivisie
5,2008/2009,858,France Ligue 1
6,2008/2009,552,Portugal Liga ZON Sagres
7,2008/2009,548,Scotland Premier League
8,2008/2009,540,Switzerland Super League
9,2008/2009,524,Poland Ekstraklasa


Being a passionate follower of Wisła Kraków, my local team, I was really happy to see them in the database. This made me want to look into their results and remember the matches. So I made a new table with just Wisła Kraków results and added a column called "Points". This was for keeping record of the points they got in each match - 3 points for a win, 1 for a draw and 0 for a loss. Wisła_Kraków table was created before, using CREATE TABLE Wisła_Kraków AS - and the following code below:

In [18]:
Wisła = pd.read_sql("""SELECT m.date, m.season, t.team_long_name AS Home_Team_Name, a.team_long_name AS Away_Team_Name, m.result, m.Score
FROM Match m
JOIN Team t
ON m.home_team_api_id = t.team_api_id
Join Team a
ON m.away_team_api_id = a.team_api_id
Where Home_Team_Name = 'Wisła Kraków' OR Away_Team_Name = 'Wisła Kraków'
Order by m.date;""", con=con)
Wisła



,date,season,Home_Team_Name,Away_Team_Name,result,Score
0,2008-08-09,2008/2009,Wisła Kraków,Polonia Bytom,1,1 - 0
1,2008-08-16,2008/2009,P. Warszawa,Wisła Kraków,2,0 - 2
2,2008-08-22,2008/2009,Wisła Kraków,GKS Bełchatów,1,2 - 1
3,2008-08-31,2008/2009,Cracovia,Wisła Kraków,0,1 - 1
4,2008-09-14,2008/2009,Wisła Kraków,Lech Poznań,2,1 - 4
...,...,...,...,...,...,...
235,2016-03-08,2015/2016,Korona Kielce,Wisła Kraków,0,1 - 1
236,2016-03-14,2015/2016,Termalica Bruk-Bet Nieciecza,Wisła Kraków,2,2 - 4
237,2016-03-18,2015/2016,Wisła Kraków,Jagiellonia Białystok,1,5 - 1
238,2016-04-03,2015/2016,Ruch Chorzów,Wisła Kraków,2,2 - 3


I altered and updated columns using this query:


Alter table Wisła_Kraków
Add column Points Numeric;

Update Wisła_Kraków
Set Points = '3'
Where home_team_goal > away_team_goal AND Home_Team_Name = 'Wisła Kraków';

Update Wisła_Kraków
Set Points = '3'
Where home_team_goal < away_team_goal AND Away_Team_Name = 'Wisła Kraków';

Update Wisła_Kraków
Set Points = '1'
Where home_team_goal = away_team_goal;

Update Wisła_Kraków
Set Points = '0'
Where home_team_goal < away_team_goal AND Home_Team_Name = 'Wisła Kraków';

Update Wisła_Kraków
Set Points = '0'
Where home_team_goal > away_team_goal AND Away_Team_Name = 'Wisła Kraków';

First, I wanted to recall the biggest wins and losses of Wisła during this time. I used aggregate functions like MIN and MAX, and used the UNION function to combine temporarily created tables.

In [19]:
Wins_and_losses = pd.read_sql("""Select date, Home_Team_Name, Away_Team_Name, Score, MAX(home_team_goal - away_team_goal) AS Diff
FROM Wisła_Kraków
UNION
Select date, Home_Team_Name, Away_Team_Name, Score, MIN(home_team_goal - away_team_goal) AS Diff
FROM Wisła_Kraków
UNION
Select date, Home_Team_Name, Away_Team_Name, Score, MAX(home_team_goal - away_team_goal) AS Diff
FROM Wisła_Kraków
WHERE Home_Team_Name <> 'Wisła Kraków'
UNION
Select date, Home_Team_Name, Away_Team_Name, Score, MIN(home_team_goal - away_team_goal) As Diff
FROM Wisła_Kraków
WHERE Away_Team_Name <> 'Wisła Kraków';""", con=con)
Wins_and_losses

,date,Home_Team_Name,Away_Team_Name,Score,Diff
0,2008-09-14,Wisła Kraków,Lech Poznań,1 - 4,-3
1,2008-09-27,Wisła Kraków,Arka Gdynia,4 - 0,4
2,2010-10-31,Lech Poznań,Wisła Kraków,4 - 1,3
3,2015-09-19,Podbeskidzie Bielsko-Biała,Wisła Kraków,0 - 6,-6


Thanks to a created "Points" column we can look for amount of wins in a certain season:

In [21]:
Wisła_wins = pd.read_sql("""Select season, points, COUNT(points)
FROM Wisła_Kraków
Group by season, points
HAVING points = 3
ORDER BY COUNT(Points) DESC;""", con=con)
Wisła_wins

,season,Points,COUNT(points)
0,2008/2009,3,19
1,2009/2010,3,19
2,2010/2011,3,17
3,2011/2012,3,12
4,2013/2014,3,12
5,2014/2015,3,11
6,2012/2013,3,10
7,2015/2016,3,8


We can already see that there has been a steep decline in Wisła's performance since the 2010/2011 season, when they won 17 times. They have only won 12 times since then, when they won the league for the last time.

In [ ]:
But did they win more games than they lose? We can also check it, using the count function again:

In [24]:
Wisła_results = pd.read_sql("""Select Points, Count(Points)
From Wisła_Kraków
Group By Points
Order by Points DESC;""", con=con)
Wisła_results

,Points,Count(Points)
0,3,108
1,1,64
2,0,68


This is probably thanks to the first three seasons of this database, but yes, Wisła won 108 games out of the 240 played in the league. This database contains only main round matches, and from the 2013/2014 season there was a change in the game system; after the main round table was split into two, Polish clubs were playing additional matches in 'master' and 'relegation' groups. In general, though, we can assume that Wisła's performance during this time was a bit better than mediocre. To be able to truly analyze the achievements of this team, we would need to have the financial data, but there wasn't any in this database.

In [25]:
Wisła_points = pd.read_sql("""Select season, SUM(Points)
From Wisła_Kraków
Group by season;""", con=con)
Wisła_points

,season,SUM(Points)
0,2008/2009,64
1,2009/2010,62
2,2010/2011,56
3,2011/2012,43
4,2012/2013,38
5,2013/2014,45
6,2014/2015,43
7,2015/2016,37


The last table of this project shows the sum of points gained in each season. As we can see in Wisła's wins table, we can come to the conclusion that the first three seasons of this database were much better than the rest. Surprisingly, Wisła won the league in the 2010/2011 season, when they gained only 56 points. That is less than two points per game (56/90), yet they still won the league by seven points clear of Śląsk Wrocław, who finished in second place.

In conclusion, this exploratory project reviewed a large dataset about European football from the 2008/2009 season to the 2015/2016 season. Through writing SQL statements and manipulating data with Pandas, I was able to analyze and show some relevant results, f.e. that statistically, playing at home increases the chances of winning. I would like to thank Kaggle for proving such an amazing dataset, and I look forward to exploring other datasets pursued for more insights about the football world. If you read this whole project I hope you had fun and I'm looking forward to hearing new ideas to analyze data using this, or any other football database.